<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Per-patient-Bland-Altman" data-toc-modified-id="Per-patient-Bland-Altman-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Per-patient Bland-Altman</a></span></li><li><span><a href="#Waveform-stats" data-toc-modified-id="Waveform-stats-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Waveform stats</a></span></li><li><span><a href="#Error-as-function-of-proximity-to-NIBP" data-toc-modified-id="Error-as-function-of-proximity-to-NIBP-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Error as function of proximity to NIBP</a></span></li><li><span><a href="#Error-as-a-function-of-Systolic-BP-variability" data-toc-modified-id="Error-as-a-function-of-Systolic-BP-variability-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Error as a function of Systolic BP variability</a></span></li><li><span><a href="#Joint-error-as-function-of-time-from-NIBP" data-toc-modified-id="Joint-error-as-function-of-time-from-NIBP-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Joint error as function of time from NIBP</a></span></li><li><span><a href="#Plotly-figure-for-debugging" data-toc-modified-id="Plotly-figure-for-debugging-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plotly figure for debugging</a></span></li></ul></div>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import sys
from tqdm import tqdm
import plotly.express as px

from sklearn.metrics import mean_squared_error, mean_absolute_error

import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

sys.path.append("../")
import src.project_configs as project_configs
import src.utils as utils

# predictions_dir = "../src/models/model_predictions_sideris_4s/"
#predictions_dir = os.path.join(project_configs.project_dir, "ppg_baseline_noise_test_patients")
predictions_dir = "/Volumes/External/mimic_v7_2s/ppg_baseline_no_noise_test_patients"

save_dir = os.path.join("../reports/figures/", 
                        os.path.basename(project_configs.project_dir), 
                        os.path.basename(predictions_dir))
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
print("Loading predictions from {}".format(predictions_dir))
print("Saving results to {}".format(save_dir))

In [ ]:
def wave_rmse(window):
    if window.shape[0] > 0:
        win = window[np.all(~window[["y_true", "y_pred"]].isna(), axis=1)]
        return np.sqrt(mean_squared_error(win["y_true"], win["y_pred"]))
    else:
        return np.nan

In [ ]:
def wave_corr(window):
    if window.shape[0] > 0:
        win = window[np.all(~window[["y_true", "y_pred"]].isna(), axis=1)]
        return np.corrcoef(win["y_true"], win["y_pred"])[0, 1]
    else:
        return np.nan

In [ ]:
def wave_mae(window):
    if window.shape[0] > 0:
        win = window[np.all(~window[["y_true", "y_pred"]].isna(), axis=1)]
        return mean_absolute_error(win["y_true"], win["y_pred"])
    else: 
        return np.nan

In [ ]:
pred_files = glob.glob(os.path.join(predictions_dir, "*.csv.gz"))

print("Found {} files".format(len(pred_files)))

results_dfs_list = []

for f in tqdm(pred_files):
    try:
        df = pd.read_csv(f, sep=",", header=0)
        results_dfs_list.append(df)
    except:
        print("Count not find any data in {}".format(f))

In [ ]:
results_df = pd.concat(results_dfs_list)
print(results_df.columns.values)
del results_dfs_list

In [ ]:
results_df.shape

# Per-patient Bland-Altman

In [ ]:
def get_beat_bp_vals(window):
    # get indices of sys/dias BP
    true_bp_max_indices, true_bp_min_indices = utils.get_art_peaks(window["y_true"])
    pred_bp_max_indices, pred_bp_min_indices = utils.get_art_peaks(window["y_pred"])

    # align bp indices in case of different number of peaks
    true_bp_max_indices, pred_bp_max_indices = utils.align_lists(true_bp_max_indices, pred_bp_max_indices)
    true_bp_min_indices, pred_bp_min_indices = utils.align_lists(true_bp_min_indices, pred_bp_min_indices)

    # get values of blood pressure at peak indices
    y_true_sys_bp_all = window.iloc[true_bp_max_indices, :]["y_true"].values
    y_true_dias_bp_all = window.iloc[true_bp_min_indices, :]["y_true"].values

    y_pred_sys_bp_all = window.iloc[pred_bp_max_indices, :]["y_pred"].values
    y_pred_dias_bp_all = window.iloc[pred_bp_min_indices, :]["y_pred"].values

    return y_true_sys_bp_all, y_pred_sys_bp_all, y_true_dias_bp_all, y_pred_dias_bp_all

In [ ]:
results_df.head()

# Waveform stats

In [ ]:
per_patient_error = results_df.groupby("patient_ID").apply(wave_rmse)
per_patient_corr = results_df.groupby("patient_ID").apply(wave_corr)
per_patient_mae = results_df.groupby("patient_ID").apply(wave_mae)

In [ ]:
# log to file
with open(os.path.join(save_dir, "wave_error_stats.txt"), "w") as stats_f:
    rmse_results = "RMSE: {} +/- {}\n".format(bs.bootstrap(per_patient_error.values, stat_func=bs_stats.mean), 
                                              bs.bootstrap(per_patient_error.values, stat_func=bs_stats.std))
    print(rmse_results)
    stats_f.write(rmse_results)
    
    corr_results = "Corr: {} +/- {}\n".format(bs.bootstrap(per_patient_corr.values, stat_func=bs_stats.mean), 
                                              bs.bootstrap(per_patient_corr.values, stat_func=bs_stats.std))
    print(corr_results)
    stats_f.write(corr_results)
    
    mae_results = "MAE: {} +/- {}\n".format(bs.bootstrap(per_patient_mae.values, stat_func=bs_stats.mean), 
                                            bs.bootstrap(per_patient_mae.values, stat_func=bs_stats.std))
    print(mae_results)
    stats_f.write(mae_results)


In [ ]:
#results_df[results_df["patient_ID"] == "p027245"][["y_true", "y_pred"]].plot()
# results_df[(results_df["patient_ID"] == "p027245") & (results_df["ecg"] > 10)]["ecg"].plot()

In [ ]:
# results_df[(results_df["patient_ID"] == "p065689") & (results_df["ecg"] > 10)]["ecg"].plot()

In [ ]:
# x = results_df.groupby(["patient_ID", "window_number"]).apply(get_sys_vals)

In [ ]:
def bland_altman_vals(y_true, y_pred):
    means = np.mean([y_true, y_pred], axis=0)
    differences = np.array(y_true) - np.array(y_pred)
    return means, differences

In [ ]:
unique_patients = results_df["patient_ID"].unique()
patient_bland_altman_sys = {p: [[], []] for p in unique_patients}
patient_bland_altman_dias = {p: [[], []] for p in unique_patients}

for idx, val in tqdm(results_df.groupby(["patient_ID", "window_number"])):
    sys_true, sys_pred, dias_true, dias_pred = get_beat_bp_vals(val)
    bland_altman_sys = bland_altman_vals(sys_true, sys_pred)
    bland_altman_dias = bland_altman_vals(dias_true, dias_pred)
    patient_bland_altman_sys[idx[0]][0].append(bland_altman_sys[0])
    patient_bland_altman_sys[idx[0]][1].append(bland_altman_sys[1])
    patient_bland_altman_dias[idx[0]][0].append(bland_altman_dias[0])
    patient_bland_altman_dias[idx[0]][1].append(bland_altman_dias[1])
    
for p, v in patient_bland_altman_sys.items():
    v[0] = np.concatenate(v[0])
    v[1] = np.concatenate(v[1])
    
for p, v in patient_bland_altman_dias.items():
    v[0] = np.concatenate(v[0])
    v[1] = np.concatenate(v[1])

In [ ]:
for p, v in patient_bland_altman_sys.items():
    print(p, v[0].shape, v[1].shape)

In [ ]:
[np.mean(m[0]) for m in patient_bland_altman_sys.values()]

In [ ]:
def bland_altman_per_patient(bland_altman_sys, bland_altman_dias, 
                             sys_axis_lim = [50, 200], dias_axis_lim = [0, 150],
                             y_label="Invasive - Predicted Arterial Pressure [mmHg]",
                             x_label="(Invasive + Predicted Arterial Pressure)/2 [mmHg]",
                             title_string = "Bland-Altman {} ABP - Validation: {} +/- {}",
                             plot_file_name="joint_bland_altman.png", 
                             log_file_name="beat_error_stats.txt"):
    # plot bland-altman for all patients, where x-value is mean of the average, y-value is mean of
    # differences (errors), x-error-bar is std. dev. of average, and y-error-bar is std. dev. of the
    # differences (errors
    x_sys = [np.mean(m[0]) for m in bland_altman_sys.values()]
    y_sys = [np.mean(m[1]) for m in bland_altman_sys.values()]
    x_error_sys = [np.std(m[0]) for m in bland_altman_sys.values()]
    y_error_sys = [np.std(m[1]) for m in bland_altman_sys.values()]

    x_dias = [np.mean(m[0]) for m in bland_altman_dias.values()]
    y_dias = [np.mean(m[1]) for m in bland_altman_dias.values()]
    x_error_dias = [np.std(m[0]) for m in bland_altman_dias.values()]
    y_error_dias = [np.std(m[1]) for m in bland_altman_dias.values()]

    # draw lines on plot for mean, SD, 2xSD
    sys_diffs = [i for sublist in [m[1] for m in bland_altman_sys.values()] for i in sublist]
    mean_sys_diff_all = np.mean(sys_diffs)
    std_sys_diff_all = np.std(sys_diffs)
    
    print("Mean diff Sys BP: {} (+/- {})".format(mean_sys_diff_all, std_sys_diff_all))
    print("Mean Absolute diff Sys BP: {} (+/- {})".format(np.mean(np.abs(sys_diffs)), np.std(np.abs(sys_diffs))))

    population_rmse_sys = [np.sqrt(np.mean(np.square(m[1]))) for m in bland_altman_sys.values()]
    print("RMSE diff Sys BP: {} (+/- {})".format(np.mean(population_rmse_sys), np.std(population_rmse_sys)))

    dias_diffs = [i for sublist in [m[1] for m in bland_altman_dias.values()] for i in sublist]
    mean_dias_diff_all = np.mean(dias_diffs)
    std_dias_diff_all = np.std(dias_diffs)
    print("Mean diff Dias BP: {} (+/- {})".format(mean_dias_diff_all, std_dias_diff_all))
    print("Mean Absolute diff Dias BP: {} (+/- {})".format(np.mean(np.abs(dias_diffs)), np.std(np.abs(dias_diffs))))

    population_rmse_dias = [np.sqrt(np.mean(np.square(m[1]))) for m in bland_altman_dias.values()]
    print("RMSE diff Dias BP: {} (+/- {})".format(np.mean(population_rmse_dias), np.std(population_rmse_dias)))

    vals = {"Mean diff Sys BP: {} (+/- {})\n": sys_diffs, 
            "Mean Abs diff Sys BP: {} (+/- {})\n": np.abs(sys_diffs), 
            "RMSE diff Sys BP: {} (+/- {})\n": population_rmse_sys, 
            "Mean diff Dias BP: {} (+/- {})\n": dias_diffs, 
            "Mean Absolute diff Dias BP: {} (+/- {})\n": np.abs(dias_diffs), 
            "RMSE diff Dias BP: {} (+/- {})\n": population_rmse_dias}
    with open(os.path.join(save_dir, log_file_name), "w") as out_f:
        for k, v in vals.items():
            txt = k.format(bs.bootstrap(np.array(v), stat_func=bs_stats.mean, iteration_batch_size=1), 
                           bs.bootstrap(np.array(v), stat_func=bs_stats.std, iteration_batch_size=1))
            print(txt)
            out_f.write(txt)        
    
    plot_lim = [-60, 60]
    
    axis_label_font_size = 14
    title_font_size = 16
    line_limits = [1, 2]
    dashes = [[20, 5], [10, 2]]


    fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    ax[0].set_ylim(plot_lim)
    ax[0].set_xlim(sys_axis_lim)
    ax[0].set_yticks(np.arange(plot_lim[0], plot_lim[1] + 1, 15))
    ax[0].set_xticks(np.arange(sys_axis_lim[0], sys_axis_lim[1] + 1, 25))
    ax[0].tick_params(labelsize=13)
    ax[0].errorbar(x_sys, y_sys, xerr=x_error_sys, yerr=y_error_sys, fmt='o', markeredgecolor='black',
                   ecolor='g', capthick=2, capsize=2)
    ax[0].set_ylabel(y_label, fontsize=axis_label_font_size)
    ax[0].set_xlabel(x_label, fontsize=axis_label_font_size)
    # add number of points to plot
    ax[0].legend(["N={}".format(len(bland_altman_sys))], loc='upper left')
    # add title
    ax[0].set_title(title_string.format("Systolic",
                                        np.round(mean_sys_diff_all, 1),
                                        np.round(std_sys_diff_all, 1)), fontsize=title_font_size)
    # add SD lines
    ax[0].axhline(mean_sys_diff_all, linestyle='-', c='black')
    for sd_limit, dash_style in zip(line_limits, dashes):
        limit_of_agreement = sd_limit * std_sys_diff_all
        lower = mean_sys_diff_all - limit_of_agreement
        upper = mean_sys_diff_all + limit_of_agreement
        for j, lim in enumerate([lower, upper]):
            ax[0].axhline(lim, linestyle='--', dashes=dash_style, c='black')


    ax[1].set_ylim(plot_lim)
    ax[1].set_xlim(dias_axis_lim)
    ax[1].set_yticks(np.arange(plot_lim[0], plot_lim[1] + 1, 15))
    ax[1].set_xticks(np.arange(dias_axis_lim[0], dias_axis_lim[1] + 1, 25))
    ax[1].tick_params(labelsize=13)
    ax[1].errorbar(x_dias, y_dias, xerr=x_error_dias, yerr=y_error_dias, fmt='o', markeredgecolor='black',
                   ecolor='g', capthick=2, capsize=2)
    ax[1].set_ylabel(y_label, fontsize=axis_label_font_size)
    ax[1].set_xlabel(x_label, fontsize=axis_label_font_size)
    # add number of points to plot
    ax[1].legend(["N={}".format(len(bland_altman_dias))], loc='upper left')
    # add title
    ax[1].set_title(title_string.format("Diastolic",
                                        np.round(mean_dias_diff_all, 1),
                                        np.round(std_dias_diff_all, 1)), fontsize=title_font_size)
    ax[1].axhline(mean_dias_diff_all, linestyle='-', c='black')
    for sd_limit, dash_style in zip(line_limits, dashes):
        limit_of_agreement = sd_limit * std_dias_diff_all
        lower = mean_dias_diff_all - limit_of_agreement
        upper = mean_dias_diff_all + limit_of_agreement
        for j, lim in enumerate([lower, upper]):
            ax[1].axhline(lim, linestyle='--', dashes=dash_style, c='black')
    plt.savefig(os.path.join(save_dir, plot_file_name))
    plt.show()
    plt.close()
    return ax

In [ ]:
bland_altman_per_patient(patient_bland_altman_sys, patient_bland_altman_dias)

# Error as function of proximity to NIBP

In [ ]:
results_df["prox"].value_counts()

In [ ]:
results_df[results_df["prox"] < 200000]["prox"].hist(bins=100)

In [ ]:
# results_df["prox"].isna().sum()

In [ ]:
# results_df["y_true"].isna().sum()

In [ ]:
# results_df[~results_df[["y_true", "y_pred"]].isna()]["y_pred"].isna().sum()

In [ ]:
def wave_rmse_mean_std(window):
    if window.shape[0] > 0:
        win = window[np.all(~window[["y_true", "y_pred"]].isna(), axis=1)]
        mean_rmse = win.groupby("patient_ID").apply(lambda x: np.sqrt(mean_squared_error(x["y_true"], x["y_pred"]))).mean()
        std_rmse = win.groupby("patient_ID").apply(lambda x: np.sqrt(mean_squared_error(x["y_true"], x["y_pred"]))).std()
#         return np.sqrt(mean_squared_error(win["y_true"], win["y_pred"]))
        return pd.Series({'mean_rmse': mean_rmse, 'std_rmse': std_rmse})
    else:
        return np.nan

In [ ]:
max_time_from_nibp = 100*60*10
bin_width = 10
bins = range(0, int(max(results_df["prox"])), max_time_from_nibp)
bins = range(0, int(max_time_from_nibp/project_configs.sample_freq), bin_width)

# only get samples that are within reasonable time range
results_df_filtered = results_df[results_df["prox"] < max_time_from_nibp]
results_df_filtered["prox"] = results_df_filtered["prox"]/project_configs.sample_freq

In [ ]:
# group data by bin 
grouped_df = results_df_filtered.groupby(pd.cut(results_df_filtered["prox"], bins=bins)).apply(wave_rmse_mean_std)

In [ ]:
grouped_df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,6))
grouped_df["mean_rmse"].plot(ax=ax)
ax.fill_between(np.arange(len(grouped_df["std_rmse"].values)), 
                 (grouped_df["mean_rmse"] - grouped_df["std_rmse"]).values, 
                 (grouped_df["mean_rmse"] + grouped_df["std_rmse"]).values, 
                alpha=0.2)

#plt.xticks(rotation=45)
#plt.locator_params(axis='x', nbins=20)
plt.xlabel("Time from most recent NIBP measurement (seconds)")
plt.ylabel("RMSE")
plt.ylim([0, 80])
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "error_vs_time_from_cuff.png"))
plt.show()

In [ ]:
# write data to file
grouped_df.index =  grouped_df.index.astype(str)
grouped_df.to_csv(os.path.join(save_dir, "error_vs_time_from_cuff.csv"), header=True, index=True)

# Error as a function of Systolic BP variability

In [ ]:
def var_diff_vals(y_true, y_pred):
    var = np.std(y_true)
#     differences = np.array(y_true) - np.array(y_pred)
    differences = np.sqrt(mean_squared_error(y_true, y_pred))
    return var, differences

In [ ]:
patient_bp_var_error_sys = {p: [[], []] for p in unique_patients}
patient_bp_var_error_dias = {p: [[], []] for p in unique_patients}

for idx, val in results_df.groupby(["patient_ID", "window_number"]):
    sys_true, sys_pred, dias_true, dias_pred = get_beat_bp_vals(val)
    
    try:
        patient_bp_var_error_sys[idx[0]][0].append(var_diff_vals(sys_true, sys_pred)[0])
        patient_bp_var_error_sys[idx[0]][1].append(var_diff_vals(sys_true, sys_pred)[1])
        patient_bp_var_error_dias[idx[0]][0].append(var_diff_vals(dias_true, dias_pred)[0])
        patient_bp_var_error_dias[idx[0]][1].append(var_diff_vals(dias_true, dias_pred)[1])
    except ValueError:
        print("No valid data for {}".format(idx))
    
# for p, v in patient_bp_var_error_sys.items():
#     v[0] = np.concatenate(v[0])
#     v[1] = np.concatenate(v[1])
    
# for p, v in patient_bp_var_error_dias.items():
#     v[0] = np.concatenate(v[0])
#     v[1] = np.concatenate(v[1])

In [ ]:
for k, v in patient_bp_var_error_sys.items():
    print(k, len(v[0]), len(v[1]))

In [ ]:
bland_altman_per_patient(patient_bp_var_error_sys, patient_bp_var_error_dias, 
                         sys_axis_lim=[-5, 10], dias_axis_lim=[-5, 10], 
                         y_label="RMSE",
                         x_label="Standard Deviation of ABP",
                         title_string=" Error vs. Variance: {} Blood Pressure",
                         plot_file_name="error_vs_variance.png", 
                         log_file_name="error_vs_variance_results.txt")

In [ ]:
patient_bp_var_error_sys.items()

In [ ]:
sys_vars = [i for sublist in [m[0] for m in patient_bp_var_error_sys.values()] for i in sublist]
sys_diffs = [i for sublist in [m[1] for m in patient_bp_var_error_sys.values()] for i in sublist]

In [ ]:
plt.scatter(sys_vars, sys_diffs)

# Joint error as function of time from NIBP

In [ ]:
dirs_to_consider = ["model_predictions_sideris_4s", 
                    "model_predictions_vnet_4s_with_noise", 
                    "ppg_baseline_noise_test_patients"]

cuff_time_error = []
for d in dirs_to_consider:
    f = os.path.join("../reports/figures/", 
                     os.path.basename(project_configs.project_dir), 
                     d, "error_vs_time_from_cuff.csv")
    print(f)
    tdf = pd.read_csv(f, sep=",", header=0, index_col=0)
    tdf.columns = ["RMSE"]
    tdf["data"] = d
    cuff_time_error.append(tdf)

In [ ]:
dataset_name_mapping = {"model_predictions_sideris_4s": "Sideris et al.", 
                        "model_predictions_vnet_4s_with_noise": "V-Net", 
                        "ppg_baseline_noise_test_patients": "Scaled PPG"}
for i in cuff_time_error:
    i["data"] = i["data"].apply(lambda x: dataset_name_mapping[x])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
for i in cuff_time_error:
    ax.plot(i["RMSE"], label=i["data"].values[0])
plt.xlabel("Time from most recent NIBP measurement (seconds)")
plt.ylabel("RMSE")
plt.ylim([0, 80])
plt.tight_layout()
plt.locator_params(axis='x', nbins=20)
plt.show()

# Plotly figure for debugging

In [ ]:
results_df.head()

In [ ]:
results_df["diff"] = results_df["y_true"] - results_df["y_pred"]

In [ ]:
results_df["bp_avg"] = (results_df["y_true"] + results_df["y_pred"])/2

In [ ]:
# fig = px.scatter(results_df, x="bp_avg", y="diff", error_x="std_dev_bp", error_y="std_dev_error",
#                      size="num_values", hover_name="patient")

In [ ]:
fig = px.scatter(results_df.sample(n=1000), x="bp_avg", y="diff", hover_name="patient_ID", hover_data=["prox", "window_number", "window_count", "date"])
fig.show()

In [ ]:
results_df[(results_df["patient_ID"] == "p056201") & 
           (results_df["window_number"] == 3) & 
           (results_df["window_count"] == 7) & 
           (results_df["date"] == "2107-01-21-19-06")][["y_true", "y_pred", "nibp_sys", "nibp_dias"]].plot()
plt.show()
results_df[(results_df["patient_ID"] == "p056201") & 
           (results_df["window_number"] == 3) & 
           (results_df["window_count"] == 7) & 
           (results_df["date"] == "2107-01-21-19-06")][["ppg"]].plot()
plt.show()

In [ ]:
utils.get_art_peaks(results_df[(results_df["patient_ID"] == "p056201") & 
           (results_df["window_number"] == 3) & 
           (results_df["window_count"] == 7) & 
           (results_df["date"] == "2107-01-21-19-06")][["y_true"]].values[:, 0])

In [ ]:
utils.get_art_peaks(results_df[(results_df["patient_ID"] == "p056201") & 
           (results_df["window_number"] == 3) & 
           (results_df["window_count"] == 7) & 
           (results_df["date"] == "2107-01-21-19-06")][["y_pred"]].values[:, 0])

In [ ]:
for k, v in results_df[results_df["diff"].abs() > 60].groupby(["patient_ID", "window_number"]):
#     v[["y_true", "y_pred"]].plot()
#     plt.show()
#     print(k, v)
    file_string = "{}-{}_{}_preprocessed_v2.npy".format(k[0], v["date"].iloc[0], int(k[1]/project_configs.max_windows_per_file))
    print(file_string)
#     data_dir = os.path.join(os.environ["HOME"], "Downloads/test_mimic_project/test_mimic_windows")
    data_dir = project_configs.test_dir
    try:
        X = np.load(os.path.join(data_dir, file_string))
        idx = k[1] % project_configs.max_windows_per_file
#         print(idx)
        print(np.mean(X[idx*project_configs.window_size:idx*project_configs.window_size+project_configs.window_size, 2:-1], axis=0))
        fig, ax = plt.subplots(2, 1, figsize=(12, 6))
        ax[1].plot(X[idx*project_configs.window_size:idx*project_configs.window_size+project_configs.window_size, 0:2])
        ax[0].plot(X[idx*project_configs.window_size:idx*project_configs.window_size+project_configs.window_size, -1])
        plt.show()
    except FileNotFoundError:
        print("Could not find", file_string)